In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import json
from fake_useragent import UserAgent
from datetime import datetime
import time
from sqlalchemy import create_engine
from collections import Counter


ua = UserAgent()
headers = {"User-Agent":ua.random}

In [ ]:
local_time = datetime.now().strftime('%Y-%m-%d')

smtv_sports_url = "https://smtv.io/program/listwithgame/default?parent=43&child=all&day_length=7&date="+local_time+"&now=Y&filter_live=Y"
smtv_entertainment_url = "https://smtv.io/program/listwithgame/default?parent=44&child=all&day_length=7&date="+local_time+"&now=Y&filter_live=Y"
smtv_game_url = "https://smtv.io/program/listwithgame/default?parent=81&child=all&day_length=7&date="+local_time+"&now=Y&filter_live=Y"
smzb_sports_url = "https://smzb.io/program/listwithgame/default?parent=1&child=all&day_length=7&date="+local_time+"&now=Y&filter_live=Y"
smzb_entertainment_url = "https://smzb.io/program/listwithgame/default?parent=2&child=all&day_length=7&date="+local_time+"&now=Y&filter_live=Y"
smzb_game_url = "https://smzb.io/program/listwithgame/default?parent=45&child=all&day_length=7&date="+local_time+"&now=Y&filter_live=Y"
kl99_sports_url = "https://kl99.tv/program/listwithgame/default?parent=1&child=all&tag=&day_length=7&date="+local_time+"&now=Y&filter_live=Y"
lj88_sports_url = "https://lj88.tv/program/listwithgame/default?parent=27&child=all&tag=&day_length=7&date="+local_time+"&now=Y&filter_live=Y&from=gg"
ggzb_game_url = "https://ggzb.io/program/listwithgame/default?parent=1&child=all&tag=&day_length=7&date="+local_time+"&now=Y&filter_live=Y&from=gg"



In [ ]:
def Get_Schedule_diff(url,Site):
    res = requests.get(url,headers=headers)
    schedule = json.loads(res.text)
    result = []
    for i in range(len(schedule["result"])):

        for j in range(len(schedule["result"][i]["data"])):

            Category = schedule["result"][i]["data"][j]["live_types_name"]
            Type = schedule["result"][i]["data"][j]["live_types_child_name"]
            Date = schedule["result"][i]["data"][j]["time"][:10]
            StartTime = schedule["result"][i]["data"][j]["time"][-8:]
            GameName = schedule["result"][i]["data"][j]["lname"]
            TeamNameA = schedule["result"][i]["data"][j]["hname"]
            TeamNameB = schedule["result"][i]["data"][j]["aname"]

            result_dict = {
                "Site":Site,
                "Category":Category,
                "Type":Type,
                "Date":Date,
                "StartTime":StartTime,
                "GameName":GameName,
                "TeamNameA":TeamNameA,
                "TeamNameB":TeamNameB
                
            }

            result.append(result_dict)

    return result

In [ ]:
user = "root"
passwd = "Pn123456"
ip = "192.168.22.110:3306"
db_name = "DB_name"
table_name = "Schedule"

def List_to_mysql(user,passwd,ip,db_name,table_name,result_list):

    engine = create_engine('mysql+mysqlconnector://'+ user +':'+ passwd +'@'+ip+'/'+ db_name +'?charset=utf8', encoding='utf-8')
    con = engine.connect() #建立連結
    
    for item in result_list:
        df = pd.DataFrame(item, index=[0])
        try:
            df.to_sql(name=table_name,con=con,if_exists='append',index=False) #假設table已存在 就自動往下加入data

        except Exception as e:
            if 'PRIMARY' in str(e):
                pass

    con.close() 
    engine.dispose()

In [ ]:
# smtv.io

Site = "smtv.io"
result_list = Get_Schedule_diff(url=smtv_sports_url,Site=Site)
List_to_mysql(user,passwd,ip,db_name,table_name,result_list)

result_list = Get_Schedule_diff(url=smtv_entertainment_url,Site=Site)
List_to_mysql(user,passwd,ip,db_name,table_name,result_list)

result_list = Get_Schedule_diff(url=smtv_game_url,Site=Site)
List_to_mysql(user,passwd,ip,db_name,table_name,result_list)


# smzb.io

Site = "smzb.io"
result_list = Get_Schedule_diff(url=smzb_sports_url,Site=Site)
List_to_mysql(user,passwd,ip,db_name,table_name,result_list)

result_list = Get_Schedule_diff(url=smzb_entertainment_url,Site=Site)
List_to_mysql(user,passwd,ip,db_name,table_name,result_list)

result_list = Get_Schedule_diff(url=smzb_game_url,Site=Site)
List_to_mysql(user,passwd,ip,db_name,table_name,result_list)
